In [5]:
!pip install -q sentence-transformers faiss-cpu transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 864.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.4 MB/s eta 0:00:00


In [17]:
# STEP 1: Data Ingest
from google.colab import files
uploaded = files.upload()

# Assuming single file upload
filename = next(iter(uploaded.keys()))
print(f"Uploaded: {filename}")



Saving Document.txt to Document (2).txt
Uploaded: Document (2).txt


In [18]:
# STEP 2: Chunk document
# Read document content
with open(filename, 'r', encoding='utf-8') as file:
    text = file.read()

# Chunking the document
chunk_size = 300
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]



In [19]:
#STEP 4: Embedding
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer("all-MiniLM-L6-v2")

chunk_embeddings = embedder.encode(chunks)


In [20]:
#STEP 5: VectorDB
import faiss
import numpy as np

dimension = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(chunk_embeddings))

In [23]:
#STEP 6: Query Processing + Retrieval + Relevancy Filter + Output
query = input("Ask a question: ")
query_embedding = embedder.encode([query])

# Search for most relevant chunk
D, I = index.search(np.array(query_embedding), k=1)
top_chunk = chunks[I[0][0]]

# Threshold filter
if D[0][0] > 1.0:
    print("I cannot answer this question as it is not relevant to the document.")

else:
    prompt = f"Answer the question based on the following text:\n\n{top_chunk}\n\nQuestion: {query}"
    response = generate_answer(prompt)
    print("\nAnswer:", response)


Ask a question: How many days does standard shipping take?

Answer: 5–7 business days
